In [23]:
#Importacion de librerias
#Libreria OpenCV
import cv2
#Libreria Numpy
import numpy as np
#Libreria de opciones de sistema
import os
#Dar importancia de mensajes
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import shutil
import glob
#Importar pandas
import pandas as pd
#Importar librerias graficas
import matplotlib.pyplot as plt
from  PIL  import Image
#Importar librerias de analisis
import seaborn as sn
from keras.preprocessing.image import ImageDataGenerator #array_to_img, img_to_array, load_img
#from rembg import remove
from sklearn.preprocessing import MinMaxScaler
import h5py
#Importar libreria de tiempo
from time import sleep

In [24]:
#Ruta de origen de .ipynb
root_path = os.path.abspath('.')
#Obtener ruta de carpeta con imagenes
images_path =root_path+"/Banana_Ripeness_Dataset_Augmented-main"
#Enlistar carpetas de imagenes
class_names = os.listdir(images_path)
#Clasificar carpetas
class_names.sort()
print(class_names)

['totally_green', 'yellow_with_brown_spots', 'yellow_with_green_tips']


In [25]:
#copiar las bananas
#Ruta de imagnes aumentadas
banana_path= root_path+"/Banana_Ripeness_Dataset_Augmented"
#Iteracion por cada clase
for class_name in class_names:
    #Formacion de ruta de clase
    cur_path = images_path + "/" + class_name
    #Creacion de ruta de lementos
    elements = os.listdir(cur_path)
    #Creacion de ruta de archivos aumentados
    os.makedirs(banana_path+"/"+ class_name, exist_ok=True)
    
    for element in elements:
        print("Copying Image - {} in Banana_Ripeness_Dataset_Augmented/{}".format(element, class_name))
        src = os.path.join(images_path + "/" + class_name, element) # origen
        dst = os.path.join(banana_path+"/" + class_name, element) 
        shutil.copy(src, dst)


Copying Image - 041_BAN_044_1.JPG in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - 288_BAN_032_2.JPG in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - totally_green_0_3007.jpg in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - 411_BAN_157_2.JPG in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - 436_BAN_182_2.JPG in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - totally_green_0_1040.jpg in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - 088_BAN_091_1.JPG in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - totally_green_0_7520.jpg in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - totally_green_0_9007.jpg in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - totally_green_0_7251.jpg in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - totally_green_0_6098.jpg in Banana_Ripeness_Dataset_Augmented/totally_green
Copying Image - totally

# Aumento de datos

In [22]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.4,
        horizontal_flip=True,
        fill_mode='nearest')

NameError: name 'ImageDataGenerator' is not defined

# Clase totalmente verde

# Clase amarilla con puntas verdes

# Clase amarilla con manchas marrones

# Filtro NLM para eliminación de ruido de imágenes

# Eliminación de fondo